In [26]:
import ee
import geopandas as gpd
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Optional
import urllib.request
from calendar import monthrange
import requests
from geobr import read_municipality


In [27]:
ee.Initialize(project="cropyieldprediction-476612")

In [ ]:
class GEEDownloader:
    
    def __init__(self, output_dir: str = "files/gee_images"):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.mapbiomas_asset = "projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_integration_v2"
        self.sentinel2_collection = "COPERNICUS/S2_SR_HARMONIZED"
    
    def download_municipality(
        self,
        shapefile_path: str,
        crop_type: int = 39,
        resolution: int = 10,
        grid_size: int = 10,
        start_date: str = "2023-01-01",
        end_date: str = "2023-12-31",
        cloud_threshold: float = 30.0,
        composite_method: str = "median",
        bands: List[str] = ["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"],
        municipality_code: Optional[str] = None,
        municipality_name: Optional[str] = None
    ) -> Path:
        
        gdf = gpd.read_file(shapefile_path)
        geometry = gdf.geometry.values[0]
        geo_json = geometry.__geo_interface__
        ee_geometry = ee.Geometry.Polygon(geo_json['coordinates'])
        
        if not municipality_code:
            municipality_code = Path(shapefile_path).stem
        if not municipality_name:
            municipality_name = municipality_code
        
        safe_name = "".join(c for c in municipality_name if c.isalnum() or c in (' ', '-', '_')).strip()
        mun_output_dir = self.output_dir / f"{municipality_code}_{safe_name}"
        mun_output_dir.mkdir(parents=True, exist_ok=True)
        
        bounds = ee_geometry.bounds().getInfo()['coordinates'][0]
        min_lon, min_lat = bounds[0]
        max_lon, max_lat = bounds[2]
        
        lon_step = (max_lon - min_lon) / grid_size
        lat_step = (max_lat - min_lat) / grid_size
        
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
        
        current = start
        while current <= end:
            year = current.year
            month = current.month
            month_end = datetime(year, month, monthrange(year, month)[1])
            if month_end > end:
                month_end = end
            
            month_start_str = current.strftime("%Y-%m-%d")
            month_end_str = month_end.strftime("%Y-%m-%d")
            
            sentinel2 = (
                ee.ImageCollection(self.sentinel2_collection)
                .filterBounds(ee_geometry)
                .filterDate(month_start_str, month_end_str)
                .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud_threshold))
            )
            
            if sentinel2.size().getInfo() == 0:
                current = month_end + timedelta(days=1)
                continue
            
            if composite_method == "median":
                composite = sentinel2.median()
            elif composite_method == "mean":
                composite = sentinel2.mean()
            elif composite_method == "first":
                composite = sentinel2.first()
            else:
                composite = sentinel2.median()
            
            rgb = composite.select(bands).clip(ee_geometry)
            
            mapbiomas = ee.Image(self.mapbiomas_asset)
            classification = mapbiomas.select(f"classification_{year}").clip(ee_geometry)
            crop_mask = classification.eq(crop_type)
            
            rgb_masked = rgb.multiply(crop_mask).updateMask(crop_mask)
            
            for i in range(grid_size):
                for j in range(grid_size):
                    tile_lon_min = min_lon + (i * lon_step)
                    tile_lon_max = min_lon + ((i + 1) * lon_step)
                    tile_lat_min = min_lat + (j * lat_step)
                    tile_lat_max = min_lat + ((j + 1) * lat_step)
                    
                    tile_geometry = ee.Geometry.Rectangle(
                        [tile_lon_min, tile_lat_min, tile_lon_max, tile_lat_max]
                    )
                    tile_geometry = tile_geometry.intersection(ee_geometry, ee.ErrorMargin(1))
                    
                    try:
                        tile_area = tile_geometry.area().getInfo()
                        if tile_area < 1e-6:
                            continue
                    except Exception:
                        continue
                    
                    tile_crop_mask = crop_mask.clip(tile_geometry)
                    has_crop = tile_crop_mask.reduceRegion(
                        reducer=ee.Reducer.sum(),
                        geometry=tile_geometry,
                        scale=resolution * 10,
                        maxPixels=1e9
                    ).getInfo()
                    
                    if not has_crop or sum(has_crop.values()) == 0:
                        continue
                    
                    tile_rgb_masked = rgb_masked.clip(tile_geometry)
                    
                    try:
                        url = tile_rgb_masked.getDownloadUrl({
                            "region": tile_geometry,
                            "scale": resolution,
                            "crs": "EPSG:4326",
                            "format": "GEO_TIFF"
                        })
                        
                        filename = f"{municipality_code}_{year}-{month:02d}_tile_{i:02d}_{j:02d}.tif"
                        filepath = mun_output_dir / filename
                        
                        urllib.request.urlretrieve(url, filepath)
                        
                        file_size = filepath.stat().st_size
                        if file_size < 1000:
                            filepath.unlink()
                            continue
                        
                    except Exception:
                        continue
            
            current = month_end + timedelta(days=1)
        
        return mun_output_dir


In [ ]:
shapefile_dir = Path("files/municipal_shapefiles")
shapefiles = list(shapefile_dir.glob("**/*.shp"))

for shp_file in shapefiles:
    print(f"Using shapefile: {shp_file}")
    
    gdf = gpd.read_file(shp_file)
    mun_code = shp_file.stem
    mun_name = shp_file.parent.name.split('_', 1)[1] if '_' in shp_file.parent.name else mun_code
    
    downloader = GEEDownloader()
    output_path = downloader.download_municipality(
        shapefile_path=str(shp_file),
        crop_type=39,
        resolution=10,
        grid_size=5,
        start_date="2023-01-01",
        end_date="2023-06-30",
        cloud_threshold=30.0,
        composite_method="median",
        bands=["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"],
        municipality_code=mun_code,
        municipality_name=mun_name
    )
    
    print(f"Download complete! Files saved to: {output_path}")
else:
    print("No shapefiles found in files/municipal_shapefiles/")


Using shapefile: files\municipal_shapefiles\4100103_Abatiá\4100103.shp
Download complete! Files saved to: files\gee_images\4100103_Abatiá
Using shapefile: files\municipal_shapefiles\4100202_Adrianópolis\4100202.shp
Download complete! Files saved to: files\gee_images\4100202_Adrianópolis
Using shapefile: files\municipal_shapefiles\4100301_Agudos do Sul\4100301.shp
